In [9]:
class TakeawayPlayer:
    def __init__(self, player_name, strategy):
        self.player_name = player_name
        self.strategy = strategy

    def move(self, board):
        return self.strategy.move(board)

In [10]:
def run_game(num_games, toothpicks, p1_strat_name, p2_strat_name):
    # Obtain the strategies for each player
    p1_strategy = TakeawayStrategy(p1_strat_name)
    p2_strategy = TakeawayStrategy(p2_strat_name)

    # We're going to collect a lot of data
    game_data = []

    # Create a board of toothpicks
    my_board = TakeawayBoard(toothpicks)
    # Create a referee to govern the board
    my_referee = TakeawayReferee()

    # Create two players; each with a unique strategy
    player_1 = TakeawayPlayer(player_name = "player_1", strategy = p1_strategy)
    player_2 = TakeawayPlayer(player_name = "player_2", strategy = p2_strategy)

    for _ in range(num_games):
        # Create the game
        my_game = TakeawayGame(board = my_board, referee = my_referee, players = [player_1, player_2])

        # Play the game
        winner, history = my_game.play(narrated = False)
        game_data.append((winner.player_name, history))

    #for winner, history in game_data:
    #    print()
    #    for turn in history:
    #        print(turn, history[turn])
    #    print("WINNER: ", winner)

    # Write to csv
    filename = "{}_{}_{}.csv".format(p1_strat_name, p2_strat_name, num_games)
    write_to_csv(game_data, filename)

In [11]:
class TakeawayBoard:
    def __init__(self, num_toothpicks = 10):
        self.state = num_toothpicks
        self.initial_size = num_toothpicks
        self.move_history = {}

    def possible_moves(self):
        return [1, 2] if self.state > 1 else [1]

    def reset(self):
        self.state = self.initial_size
        self.move_history = {}

In [12]:
class TakeawayReferee:
    def __init__(self):
        self.is_game_over = False

    def update(self, player, board) -> int:
        current_move = self.ask_for_move(player, board)
        
        board.move_history[board.state] = {"name": player.player_name, "move": current_move }
        board.state -= current_move

        return current_move

    def ask_for_move(self, player, board) -> int:
        # To check for repeats
        deciding = True

        while deciding:
            # See what move the player would want to make.
            proposed_move = player.move(board)

            # Check to see if player has given up
            if proposed_move == 0 or proposed_move is None:
                deciding = False
                proposed_move = 0

            # No change in is_turn_over, so we should re-enter the while loop
            elif self.is_legal(proposed_move, board):
                deciding = False

        return proposed_move

    def is_legal(self, move, board) -> bool:
        return ((move > 0) and (move < 3) and (board.state - move >= 0))
    
    def check_for_winner(self, move, board, player, opponent) -> TakeawayPlayer:
        winner = None
        if move == 0 or move is None:
            winner = opponent
        elif board.state == 0:
            winner = player

        return winner

In [13]:
class TakeawayGame:
    def __init__(self, board = None, referee = None, players = None):
        self.referee = referee
        self.board = board
        self.players = players

    def play(self,narrated = False):
        self.board.reset()
        turn = 0
        winner = None

        while winner is None:
            player = self.players[turn % 2]
            opponent = self.players[(turn + 1) % 2]
            turn = turn + 1

            move = self.referee.update(player, self.board)
            winner = self.referee.check_for_winner(move, self.board, player, opponent)

            if (narrated):
                print(player.player_name, "drew", move, "toothpicks.", self.board.state, "left")
                
        return winner, self.board.move_history

In [14]:
def write_to_csv(game_data, filename):
    # Get the total number of toothpicks at the start
    start_val = max(game_data[0][1])
    # Make a descending list of all toothpicks left
    toothpicks_left = list(range(start_val, 0, -1))

    # Create our headings: Toothpicks left and turn
    headings = []
    for heading in toothpicks_left:
        headings.append(heading)
        headings.append("turn_{}".format(heading))
    headings.append("winner")

    # Start building rows; one row per game
    rows = []
    for winner, history in game_data:
        
        # How many toothpicks were taken at each turn in the game
        turns = [history[turn]["move"] for turn in history]
        # Who took those toothpicks
        names = [history[turn]["name"] for turn in history]
        
        # Start creating a row
        moves = []
        for i in range(len(turns)):
            # Append the toothpicks taken
            moves.append(turns[i])
            # Append the player who took them
            moves.append(names[i])

            # If a turn was 2, add a turn of 0 after it
            # This ensures that our rows are all the same length
            if turns[i] == 2:
                moves.append(None)
                moves.append(None)

        # Append the winner of the game
        moves.append(winner)

        # Add the row we just made to the running list of rows
        rows.append(moves)

    # Write to csv
    import csv
    with open(filename, "w") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(headings)
        csvwriter.writerows(rows)

In [15]:
from random import choice

class TakeawayStrategy:
    def __init__(self, name = "random", data = None, bias = None):
        self.name = name
        self.data = data
        self.bias = bias
        self.strategies = {"random": self.random_move,
                           "take_one": self.always_take_one,
                           "take_two": self.always_take_two,
                           "smart": self.smart_move,
                           "human": self.human
                          }
        
    def move(self, board):
        deciding = True
        possible_moves = board.possible_moves()
        moves_tried = []
        attempts = 1
        move_to_try = 0
        while deciding:
            # Get a move
            move_to_try = self.strategies[self.name](board, possible_moves)
            
            # If the move is invalid, note it and re-loop
            # Otherwise, end the loop
            if (board.state - move_to_try) < 0:
                moves_tried.append(move_to_try)
                attempts += 1
            else:
                deciding = False
            
            # If all possible moves have been tried or 3 attempts have been made
            if set(moves_tried) == set(possible_moves) or attempts == 3:
                deciding = False
                move_to_try = 0

        return move_to_try
    
    def smart_move(self, board, possible_moves):
        # Get the otpimal move
        move_to_try = 1 if self.data["Take 1 Win %"][board.state] > self.data["Take 2 Win %"][board.state] else 2
        
        # If both chances are equal, choose randomly or according to a bias, if supplied
        if self.data["Take 1 Win %"][board.state] == self.data["Take 2 Win %"][board.state]:
            move_to_try = choice(possible_moves) if self.bias is None else self.bias

        # If the move is not legal, mark it as tried and pick the only other option
        if (board.state - move_to_try) < 0:
            possible_moves.remove(move_to_try)
            move_to_try = possible_moves[0]

        # If the only other move available is invalid, return None
        if (board.state - move_to_try) < 0:
            return 0

        return move_to_try

    def random_move(self, board, possible_moves):
        # Get a random move
        return choice(possible_moves)

    def always_take_one(self, board, possible_moves):
        return 1

    def always_take_two(self, board, possible_moves):
        return 2

    def human(self, board, possible_moves):
        return int(input("Please make your move > "))

In [16]:
run_game(10, 10, "random", "random")